In [133]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os
import urllib.parse

### 게시판 검색어 입력

In [134]:
input_query = str(input())

### 검색어로 조회된 모든 데이터 크롤하기
1. url 이동
2. 셀레니움 이용해 스크롤 맨 아래로 이동
3. beautiful soup를 활용해 전체 parsing해 html파일로 저장하기
4. 저장된 html에서 제목, 조회수, 좋아요, 댓글, 링크 가져오기

In [ ]:
# Create a custom user agent string
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"

# 유저 에이전트 지정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('--user-agent=%s' % user_agent)
driver = webdriver.Chrome('./chromedriver.exe', options=options)

#0. 경로 이동
PATH = r'C:\Crawling_work\blind'
os.chdir(PATH)

#1. 검색어 점심시간 url 방문
query = input_query
encoded_query = urllib.parse.quote(query)

url = 'https://www.teamblind.com/kr/search/' + encoded_query

driver.get(url)
time.sleep(3)

#2. 맨 아래로 내리기
#스크롤 내리기 이동 전 위치
scroll_location = driver.execute_script("return document.body.scrollHeight")

while True:
    # 현재 스크롤 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 스크롤 다운 실행
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 일정 시간 대기
    time.sleep(2)

    # 새로운 스크롤 높이 저장
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 스크롤 높이와 이전 스크롤 높이가 같으면 while문 종료
    if new_height == last_height:
        break

# 3. bs4로 파싱해서 soup에 저장
# bs4로 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

with open(f'블라인드_{input_query}.html', 'w', encoding='utf-8') as f:
    f.write(str(soup))
driver.close()

# 저장된 html에서 정보 가져오기
### title, 링크, 조회수, 좋아요, 댓글수, 작성시간 가져오기

In [143]:
title_list = soup.select('div.article-list div.tit h3')
sub_list = soup.select('div.wrap-info')

temp_title =[]
temp_link = []
temp_view = []
temp_like = []
temp_comment_cnt = []
temp_date = []

# 전체에서
for i in range(len(title_list)):
    # 제목 집어넣기
    temp_title.append(title_list[i].text.strip())
    # 링크 집어넣기
    url = 'https://www.teamblind.com' + title_list[i].find('a', class_='')['href']
    temp_link.append(url)
    
    # 조회수
    temp_view.append(sub_list[i].text.split()[0])
    # 좋아요
    temp_like.append(sub_list[i].text.split()[1])
    # 댓글수
    temp_comment_cnt.append(sub_list[i].text.split()[2])
    # 작성시간
    temp_date.append(sub_list[i].text.split()[3])

In [144]:
df = pd.DataFrame(zip(temp_title, temp_link, temp_view, temp_like, temp_comment_cnt, temp_date))

In [145]:
# 2023년 3월 18일 기준 크롤링
df

,0,1,2,3,4,5
0,앱테크,https://www.teamblind.com/kr/post/%EC%95%B1%ED...,"조회수1,631",좋아요좋아요,댓글11,작성시간2021.05.02.
1,앱테크,https://www.teamblind.com/kr/post/%EC%95%B1%ED...,조회수820,좋아요1,댓글7,작성시간2021.07.10.
2,앱테크 뭐해??,https://www.teamblind.com/kr/post/%EC%95%B1%ED...,조회수306,좋아요좋아요,댓글11,작성시간01.30
3,앱테크 관련,https://www.teamblind.com/kr/post/%EC%95%B1%ED...,조회수128,좋아요좋아요,댓글1,작성시간01.18
4,짠테크?앱테크? 뭐가 있을까요,https://www.teamblind.com/kr/post/%EC%A7%A0%ED...,조회수822,좋아요좋아요,댓글2,작성시간2022.05.31.
...,...,...,...,...,...,...
203,2021-12-28 금일 한경 신문 요약(사견추가),https://www.teamblind.com/kr/post/2021-12-28-%...,"조회수1,597",좋아요46,댓글8,작성시간2021.12.28.
204,2022-01-19 금일 한경 신문 요약(사견추가),https://www.teamblind.com/kr/post/2022-01-19-%...,"조회수1,572",좋아요48,댓글14,작성시간2022.01.19.
205,현재 국내 퍼블 & 그외 일본겜들,https://www.teamblind.com/kr/post/%ED%98%84%EC...,"조회수1,645",좋아요9,댓글22,작성시간2022.09.04.
206,20180327 주요뉴스 브리핑,https://www.teamblind.com/kr/post/20180327-%EC...,"조회수4,065",좋아요12,댓글4,작성시간2018.03.27.


In [140]:
df.columns = ['제목', '링크', '조회수', '좋아요', '댓글','작성시간']
df.to_csv(f'블라인드_{input_query}.csv', encoding='utf-8-sig')